<a href="https://colab.research.google.com/github/hassanSattariNia/FederatedLearning/blob/main/split__model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install transformers datasets torch tqdm scikit-learn

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.8 MB/s eta 0:00:00


In [ ]:
from transformers import AlbertModel , AlbertConfig

In [ ]:
model = AlbertModel.from_pretrained("albert-base-v2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/47.4M [00:00<?, ?B/s]

In [ ]:
model.encoder.albert_layer_groups

ModuleList(
  (0): AlbertLayerGroup(
    (albert_layers): ModuleList(
      (0): AlbertLayer(
        (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (attention): AlbertAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Linear(in_features=768, out_features=768, bias=True)
          (attention_dropout): Dropout(p=0, inplace=False)
          (output_dropout): Dropout(p=0, inplace=False)
          (dense): Linear(in_features=768, out_features=768, bias=True)
          (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        )
        (ffn): Linear(in_features=768, out_features=3072, bias=True)
        (ffn_output): Linear(in_features=3072, out_features=768, bias=True)
        (activation): NewGELUActivation()
        (dropout): Dropout(p=0, inplace=False)
      )
    )
  )
)

In [ ]:
print(model)

AlbertModel(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(30000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0, inplace=False)
      

In [ ]:
from transformers import AlbertModel
import torch.nn as nn

model = AlbertModel.from_pretrained("albert-base-v2")

class AlbertPart1(nn.Module):
    def __init__(self, model):
        super(AlbertPart1, self).__init__()
        self.embeddings = model.embeddings
        self.encoder = model.encoder

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        embedding_output = self.embeddings(input_ids=input_ids, token_type_ids=token_type_ids)
        encoder_output = self.encoder(embedding_output, attention_mask=attention_mask)
        return encoder_output

class AlbertPart2(nn.Module):
    def __init__(self, model):
        super(AlbertPart2, self).__init__()
        self.pooler = model.pooler
        self.pooler_activation = model.pooler_activation

    def forward(self, encoder_output):
        pooled_output = self.pooler(encoder_output[0])
        pooled_output = self.pooler_activation(pooled_output)
        return pooled_output

part1 = AlbertPart1(model)
part2 = AlbertPart2(model)


In [ ]:
print(part1)

AlbertPart1(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(30000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0, inplace=False)
      

In [ ]:
print(part2)

AlbertPart2(
  (pooler): Linear(in_features=768, out_features=768, bias=True)
  (pooler_activation): Tanh()
)


In [ ]:
from transformers import AlbertTokenizer


tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")


sentence1 = "Hello, how are you?"
sentence2 = "I'm fine, thank you."


inputs = tokenizer(sentence1, sentence2  , return_tensors="pt")


print("Input IDs:", inputs["input_ids"])
print("Attention Mask:", inputs["attention_mask"])
print("Token Type IDs:", inputs["token_type_ids"])


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Input IDs: tensor([[    2, 10975,    15,   184,    50,    42,    60,     3,    31,    22,
            79,  1123,    15,  3531,    42,     9,     3]])
Attention Mask: tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
Token Type IDs: tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]])
